<a href="https://colab.research.google.com/github/BluBloos/3D-Hand-Tracking/blob/main/src/HandTracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP

In [13]:
# RUN THIS BLOCK ONLY WHEN IN COLAB

!echo "Initializing github repository"
!ls -la
!rm -r .config/
!rm -r sample_data/
!git clone https://github.com/BluBloos/QMIND2021-2022/ .

Initializing github repository
total 56
drwxr-xr-x 1 root root 4096 Mar 31 22:38 .
drwxr-xr-x 1 root root 4096 Mar 31 21:50 ..
drwxr-xr-x 1 root root    0 Mar 31 22:41 data
drwxr-xr-x 8 root root 4096 Mar 31 22:38 .git
-rw-r--r-- 1 root root  172 Mar 31 21:57 .gitignore
drwxr-xr-x 3 root root 4096 Mar 31 21:57 MessyCloset
drwxr-xr-x 2 root root 4096 Mar 31 21:57 paper
-rw-r--r-- 1 root root 2308 Mar 31 21:57 README.md
drwxr-xr-x 4 root root 4096 Mar 31 21:57 RHD_small
-rwxr-xr-x 1 root root   30 Mar 31 21:57 run.sh
drwxr-xr-x 4 root root 4096 Mar 31 21:57 src
-rw-r--r-- 1 root root 4460 Mar 31 21:57 TODO.md
drwxr-xr-x 2 root root 4096 Mar 31 21:57 .vscode
rm: cannot remove '.config/': No such file or directory
rm: cannot remove 'sample_data/': No such file or directory
fatal: destination path '.' already exists and is not an empty directory.


In [14]:
# ALWAYS RUN THIS BLOCK, COLAB OR NOT

# Download updated project from Github.
!git pull

##### HANDLE DIFFS WHEN RUNNING IN COLAB #####
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
print("In Colab:", IN_COLAB)
import sys
if (IN_COLAB):
  sys.path.insert(1, '/content/src/')
##### HANDLE DIFFS WHEN RUNNING IN COLAB #####

########### TEST GPU AND RAM OF COLLAB INSTANCE ###########
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')
########### TEST GPU AND RAM OF COLLAB INSTANCE ###########

######### EXTERNAL LIBRARIES #########
import os
import pickle
import matplotlib.pyplot as plt
import imageio
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, UpSampling2D, MaxPool2D
from tensorflow.keras import Model
print("TensorFlow version:", tf.__version__)
#NOTE: Good resource. -> https://www.tensorflow.org/tutorials/quickstart/advanced
import cv2 # opencv, for image resizing.
!pip install chumpy
######### EXTERNAL LIBRARIES #########

############## HELPER FUNCTIONS ############## 
# NOTE(Noah): Stole this function from Stackoverflow :)
def rgb2gray(rgb):
    return np.expand_dims(np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140]), axis=2)
def resize(img, size):
    return cv2.resize(img, dsize=(size, size), interpolation=cv2.INTER_CUBIC)
############## HELPER FUNCTIONS ############## 

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/BluBloos/QMIND2021-2022
   b35791d..5cfb611  main       -> origin/main
Updating b35791d..5cfb611
Fast-forward
 src/HandTracking.ipynb | 1827 ++----------------------------------------------
 1 file changed, 73 insertions(+), 1754 deletions(-)
In Colab: True
Not connected to a GPU
Your runtime has 27.3 gigabytes of available RAM
You are using a high-RAM runtime!
TensorFlow version: 2.8.0
     |████████████████████████████████| 50 kB 4.6 MB/s 
  Created wheel for chumpy: filename=chumpy-0.70-py3-none-any.whl size=58285 sha256=f156f71c82f580b05019091faef3a5291f761c009007a8a6962338b97f972a05
  Stored in directory: /root/.cache/pip/wheels/59/68/de/5e0c5d77e573e8c150e69e07a25035e6b6a04952d6e1814dbc
Successfully built chumpy


# DATA LOADING

In [15]:
from matplotlib.image import imread

# NOTE(Noah): gcs code will on work on the colab as this code is Ubuntu specific.
# I attempted to install gcsfuse on my macOS machine, but I have a version that is too new.
# also, gcsfuse simply does not work on Windows.
#
# gcsfuse is actually beta software.
if IN_COLAB:
  from google.colab import auth
  auth.authenticate_user()

  # we know that we are on an Ubuntu machine.
  # Thus, installing gcsfuse will be done via the Ubuntu instructions.
  # https://github.com/GoogleCloudPlatform/gcsfuse/blob/master/docs/installing.md#ubuntu-and-debian-latest-releases
  !echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
  !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
  
  # -y in apt-get will assume "yes" as the answer to all prompts.
  # -q in apt-get will make things "quiet" for us. Nice!
  !sudo apt-get -y -q update
  !sudo apt-get -y -q install gcsfuse

  !mkdir -p data
  !gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 shd_final data

def download_image(path):
  image = imageio.imread(path)
  _image = image.astype('float32')
  if GRAYSCALE:
      _image = rgb2gray(_image / 255)
  else:
      _image = _image / 255
  _image = resize(_image, IMAGE_SIZE)
  return _image

# TODO(Noah): Reimplement the code that sets up SH_RHD.
gcs_path = 'data' if IN_COLAB else os.path.join("..", "SH_RHD")
train_list = os.listdir(os.path.join(gcs_path, "training/color"))
eval_list = os.listdir(os.path.join(gcs_path, "evaluation/color"))

# Setup some params.
IMAGE_SIZE = 224
GRAYSCALE = False
IMAGE_CHANNELS = 1 if GRAYSCALE else 3
BATCH_SIZE = 32


# numpy "buckets" that we will use to load things in.
x_train = np.zeros( (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS) )
y_train = np.zeros( (BATCH_SIZE, 21, 3) )
x_test = np.zeros( (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS) ) 
y_test = np.zeros( (BATCH_SIZE, 21, 3) )
TRAIN_IMAGES = (len(train_list) // BATCH_SIZE) * BATCH_SIZE
TEST_IMAGES = (len(eval_list) // BATCH_SIZE) * BATCH_SIZE

deb http://packages.cloud.google.com/apt gcsfuse-bionic main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0   117k      0 --:--:-- --:--:-- --:--:--  117k
OK
Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://packages.cloud.google.com/apt gcsfuse-bionic InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-upda

In [65]:
from numpy.core.arrayprint import array_str

anno_train_path = '/content/data/anno/anno_training.pickle'
anno_eval_path = '/content/data/anno/anno_evaluation.pickle'



def load_anno(path, arr):
  anno_all = []
  count = 0
  with open(path, 'rb') as f:
    anno_all = pickle.load(f)

  for key, value in anno_all.items():
    if(count>BATCH_SIZE):
      break
    kp_visible = (value['uv_vis'][:, 2] == 1)
    case1 = np.sum(kp_visible[0:21])
    case2 = np.sum(kp_visible[21:])
    valid_case = (case1 > 0 and case2 == 0) or (case1 == 0 and case2 > 0) 
    if(valid_case):
      if(case1 ==0):
        arr[count,:,:]= value['xyz'][21:42]
        count+=1
      if(case2 == 0):
        arr[count,:,:]= value['xyz'][:21]
        count+=1


load_anno(anno_train_path, y_test)
load_anno(anno_eval_path, y_train)
print(y_test)
print(y_train)

[[[-0.05232     0.04117     0.50650001]
  [-0.0145      0.05161     0.38370001]
  [-0.02626     0.03721     0.4012    ]
  ...
  [-0.03639     0.1207      0.44679999]
  [-0.02022     0.124       0.44850001]
  [-0.01898     0.1031      0.46970001]]

 [[ 0.01842    -0.1045      0.36899999]
  [-0.03311    -0.03275     0.4533    ]
  [-0.02735    -0.04798     0.43669999]
  ...
  [ 0.03712     0.001316    0.41170001]
  [ 0.03799    -0.01236     0.40369999]
  [ 0.03472    -0.03851     0.40130001]]

 [[ 0.06759    -0.2158      0.57309997]
  [ 0.09856     0.003095    0.50629997]
  [ 0.1105     -0.06151     0.51679999]
  ...
  [-0.007587   -0.01352     0.52329999]
  [-0.009917   -0.03699     0.54650003]
  [-0.005691   -0.09856     0.56379998]]

 ...

 [[ 0.07942     0.05822     0.3971    ]
  [-0.06131     0.05076     0.39039999]
  [-0.02952     0.05302     0.3863    ]
  ...
  [-0.04179     0.03308     0.45429999]
  [-0.02578     0.02308     0.44850001]
  [ 0.00719     0.01256     0.4413    ]]

 [

# MODEL LOADING

In [16]:
# TODO(Noah): Get the MANO folders hosted in GCS so that this works again.
#   We note that this cost was tested and is in full working order, so 
#   the only thing not working is the lack of existence of MANO_DIR. 

MANO_DIR = os.path.join("data", "mano_v1_2") if IN_COLAB else os.path.join("..", "mano_v1_2")

from mobilehand import MAKE_MOBILE_HAND
from mobilehand_lfuncs import LOSS_3D

MOBILE_HAND = MAKE_MOBILE_HAND(IMAGE_SIZE, IMAGE_CHANNELS, BATCH_SIZE, MANO_DIR)

# INTEGRATION TEST
input_test = tf.random.uniform(shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS))
input_test = tf.cast(input_test, tf.float32)
output_test = MOBILE_HAND(input_test)
print(output_test)

# The lower training loop assumes that the model is set as such.
model = MOBILE_HAND

# The lower training loop also assumes that we have the loss function set like so.
loss_fn = lambda pred, gt : LOSS_3D(pred,gt) 

ValueError: ignored

# TRAINING LOOP

In [ ]:
class StupidSimpleLossMetric():
    def __init__(self):
        self.losses = [] # empty python array 
    def __call__(self, loss):
        self.losses.append(loss)
    def result(self):
        return sum(self.losses) / len(self.losses)
    def reset_states(self):
        self.losses = []

optimizer = tf.keras.optimizers.Adam() # defaults should work just fine
train_loss = StupidSimpleLossMetric()
test_loss = StupidSimpleLossMetric()

# Loss function unit test
input = tf.zeros([1, 21,3])  # mock pred of all zeros
label = np.expand_dims(y_train[0], axis=0)
loss = loss_fn(input, label) 
print('Loss for pred of all zeros', loss.numpy())
#loss2 = loss_fn(label, label)
#print('Loss for perfect prediction', loss2.numpy())
input2 = tf.ones([1, 21, 3])
loss3 = loss_fn(input2, label)
print('Loss for pred of all ones', loss3.numpy())

@tf.function
def train_step(input, gt):
    with tf.GradientTape() as tape:
        predictions = model(input)
        #loss = loss_func(predictions, segmentation_masks)
        #loss = np.dot(tf.reshape(segmentation_masks, [102400], tf.reshape(predictions, [102400])
        loss = loss_fn(predictions, gt)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss
    #train_accuracy(labels, predictions)
  
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  return loss_fn(predictions, labels)
  #test_accuracy(labels, predictions)

In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
def cstr(str): # cyan string
    return bcolors.OKCYAN + str + bcolors.ENDC

checkpoint_path = "checkpoints/" if IN_COLAB else os.path.join("..", "checkpoints/")
!mkdir $checkpoint_path

last_checkpoint = -1
if (last_checkpoint > -1):
  file_path = os.path.join(checkpoint_path, "cp-{:04d}.ckpt".format(last_checkpoint))
  model.load_weights(file_path)
  print(cstr("Loaded weights from {}".format(file_path)))

EPOCHS = 10 # sure...

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  print("Begin epoch", epoch)
  start = time.time()
  train_loss.reset_states()
  #train_accuracy.reset_states()
  test_loss.reset_states()
  #test_accuracy.reset_states()
  
  for i in range(TRAIN_IMAGES // 32):
    for j in range(32):
      train_image = download_image(os.path.join(gcs_path, "training", "color", train_list[j + i * 32]))
      x_train[j,:,:,:] = train_image
     
    # TODO(Data Team): Load in the annotations.
    x_train = x_train.astype('float32')
    y_train = y_train.astype('float32')

    loss = train_step(x_train, y_train)
    train_loss(loss.numpy())

  for i in range(TEST_IMAGES // 32):
    for j in range(32):
      eval_image = download_image(os.path.join(gcs_path, "evaluation", "color", eval_list[j + i * 32]))
      x_test[j,:,:,:] = eval_image
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')

    loss_test = test_step(x_test, y_test)
    test_loss(loss.numpy())

  end = time.time()

  print(
    f'Epoch {epoch}, '
    f'Time {end-start} s'
    f'Loss: {train_loss.result()}, '
    f'Test Loss: {test_loss.result()}, '
  )

  # Save the model parameters
  if (epoch % 5 == 0):

    checkpoint_filepath = os.path.join(checkpoint_path, "cp-{:04d}.ckpt".format(epoch))
    model.save_weights(checkpoint_filepath)
    print(cstr("Saved weights to {}".format(checkpoint_filepath)))